# side model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import SIDE

df_features = pd.read_parquet("data/features.parquet")

features = df_features[SIDE]
label = np.load("data/side_label.npy")
len_gap = len(label) - len(df_features)
label = label[len_gap:]

print(features.shape)
print(label.shape)

mask = features.index < date_to_timestamp("2025-01-01")
features_masked = features[mask]
label_masked = label[mask]

print(features_masked.shape)
print(label_masked.shape)
print(np.unique(label_masked, return_counts=True))

features.isna().sum().sort_values(ascending=False)

(34487, 454)
(34487,)
(31933, 454)
(31933,)
(array([0, 1]), array([14430, 17503]))


williams_r                   0
mod_rsi                      0
hasbrouck_lambda_lag10       0
chaiken_money_flow_lag1      0
swamicharts_rsi_1            0
                            ..
swamicharts_stochastic_43    0
swamicharts_stochastic_9     0
reactivity_lag2              0
roofing_filter_dt_lag2       0
hurst_coef_fast_lag11        0
Length: 454, dtype: int64

In [2]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "metric": "auc",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(features_masked, label_masked)
res = lgb.cv(params, dtrain, num_boost_round=100, nfold=5, stratified=True)
res.keys()

dict_keys(['valid auc-mean', 'valid auc-stdv'])

In [3]:
res["valid auc-mean"][-1]

0.9101366058458827

In [2]:
import lightgbm as lgb
import optuna


def objective(trial):
    METRIC = "auc"

    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(features, label)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        metrics=METRIC,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-18 23:17:30,521] A new study created in memory with name: no-name-2db58c38-2269-4f99-aace-c4ba014f5b31
[I 2025-06-18 23:17:53,316] Trial 0 finished with value: 0.9559844019193159 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 115, 'max_depth': 386, 'min_gain_to_split': 0.4200826773675722, 'min_data_in_leaf': 439, 'lambda_l1': 69.748583676912, 'lambda_l2': 28.425183642022915, 'num_boost_round': 608}. Best is trial 0 with value: 0.9559844019193159.
[I 2025-06-18 23:18:06,366] Trial 1 finished with value: 0.9507230057373002 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 137, 'max_depth': 361, 'min_gain_to_split': 0.12182442282017833, 'min_data_in_leaf': 262, 'lambda_l1': 97.95566749490527, 'lambda_l2': 60.33131323770954, 'num_boost_round': 392}. Best is trial 0 with value: 0.9559844019193159.
[I 2025-06-18 23:18:15,763] Trial 2 finished with value: 0.9614500181752001 and parameters: {'

In [3]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 215,
 'max_depth': 949,
 'min_gain_to_split': 0.25359262683421413,
 'min_data_in_leaf': 452,
 'lambda_l1': 2.330642930450268,
 'lambda_l2': 21.688550291609964,
 'num_boost_round': 743}

In [4]:
import lightgbm as lgb  # noqa

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(features_masked, label_masked)
side_model = lgb.train(params, dtrain)
side_model_prod = lgb.train(params, lgb.Dataset(features, label))

In [5]:
import plotly.express as px

test_features = features[features.index > date_to_timestamp("2025-01-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [6]:
side_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_side.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcEntropyBarV1/model/model_side_prod.txt"
)

# meta model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import (
    META_ALL,
    SIDE,
    get_side_model,
)

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = get_side_model(False)

side_model_res = side_model.predict(df_features[SIDE])

df_features["model"] = side_model_res

meta_features = df_features[META_ALL]
print(meta_features.shape)
print(meta_label.shape)

mask = meta_features.index < date_to_timestamp("2025-01-01")
meta_features_masked = meta_features[mask]
meta_label_masked = meta_label[mask]

print(meta_features_masked.shape)
print(meta_label_masked.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(34487, 916)
(34487,)
(31933, 916)
(31933,)


cwt_11_ddt_lag2                     0
adaptive_bp_dt_lag15                0
chaiken_money_flow_ddt_lag2         0
cwt_2_lag12                         0
entropy_for_jesse_ddt_lag17         0
                                   ..
adaptive_stochastic_ddt_lag1        0
vwap_dt_lag6                        0
roll_impact_dt_lag1                 0
norm_on_balance_volume_ddt_lag19    0
model                               0
Length: 916, dtype: int64

In [2]:
import lightgbm as lgb
from sklearn.metrics import (
    f1_score,  # noqa
    fbeta_score,  # noqa
)

METRIC = "f1"


def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="macro")
    higher_better = True
    return metric_name, value, higher_better


params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(meta_features, meta_label)
res = lgb.cv(
    params, dtrain, num_boost_round=100, nfold=5, stratified=True, feval=eval_metric
)
res.keys()

dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv', 'valid f1-mean', 'valid f1-stdv'])

In [4]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features, meta_label)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        stratified=True,
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(n_startup_trials=100),
)
study.optimize(objective, n_trials=500, n_jobs=1)

[I 2025-06-19 00:10:34,951] A new study created in memory with name: no-name-9ee21d0b-1744-4462-9e22-91a03f26a2ac
[I 2025-06-19 00:11:34,399] Trial 0 finished with value: 0.6833384954828481 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 136, 'max_depth': 510, 'min_gain_to_split': 0.16861338752885185, 'min_data_in_leaf': 197, 'lambda_l1': 0.2937736485703152, 'lambda_l2': 27.178446365842394, 'num_boost_round': 491}. Best is trial 0 with value: 0.6833384954828481.
[I 2025-06-19 00:11:47,663] Trial 1 finished with value: 0.5547281631375769 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 148, 'max_depth': 658, 'min_gain_to_split': 0.5437461660719761, 'min_data_in_leaf': 211, 'lambda_l1': 86.61594363527128, 'lambda_l2': 33.823353559965, 'num_boost_round': 318}. Best is trial 0 with value: 0.6833384954828481.
[I 2025-06-19 00:13:18,336] Trial 2 finished with value: 0.6495071574107621 and parameters:

KeyboardInterrupt: 

In [5]:
print("meta model: ")
study.best_params

meta model: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 83,
 'max_depth': 317,
 'min_gain_to_split': 0.021882454016651367,
 'min_data_in_leaf': 148,
 'lambda_l1': 1.7860860481034284,
 'lambda_l2': 68.31336339690397,
 'num_boost_round': 876}

In [6]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_masked, meta_label_masked),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [7]:
import plotly.express as px

res = model.predict(
    meta_features[meta_features.index > date_to_timestamp("2025-01-01")]
)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta.txt")
prod_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta_prod.txt")